In [18]:
import numpy as np
import cvxopt #用于求解线性规划
from process_data import load_and_process_data
from evaluation import get_micro_F1,get_macro_F1,get_acc


#根据指定类别main_class生成1/-1标签
def svm_label(labels,main_class):
    new_label=[]
    for i in range(len(labels)):
        if labels[i]==main_class:
            new_label.append(1)
        else:
            new_label.append(-1)
    return np.array(new_label)

# 实现线性回归
class SupportVectorMachine:

    '''参数初始化 
    lr: 梯度更新的学习率
    Lambda: L2范数的系数
    epochs: 更新迭代的次数
    '''
    def __init__(self,kernel,C,Epsilon):
        self.kernel=kernel
        self.C = C
        self.Epsilon=Epsilon

    '''KERNEL用于计算两个样本x1,x2的核函数'''
    def KERNEL(self, x1, x2, kernel='Gauss', d=2, sigma=1):
        #d是多项式核的次数,sigma为Gauss核的参数
        K = 0
        if kernel == 'Gauss':
            K = np.exp(-(np.sum((x1 - x2) ** 2)) / (2 * sigma ** 2))
        elif kernel == 'Linear':
            K = np.dot(x1,x2)
        elif kernel == 'Poly':
            K = np.dot(x1,x2) ** d
        else:
            print('No support for this kernel')
        return K

    '''
    根据训练数据train_data,train_label（均为np数组）求解svm,并对test_data进行预测,返回预测分数，即svm使用符号函数sign之前的值
    train_data的shape=(train_num,train_dim),train_label的shape=(train_num,) train_num为训练数据的数目，train_dim为样本维度
    预测结果的数据类型应为np数组，shape=(test_num,1) test_num为测试数据的数目
    '''
    def fit(self,train_data,train_label,test_data):
        #需要你实现的部分
        train_num,features = train_data.shape
        kernel = np.zeros((train_num,train_num))
        if self.kernel == 'Linear':
            kernel = train_data@train_data.T
        else:
            for i in range(train_num):
                for j in range(train_num):
                    kernel[i,j] = self.KERNEL(train_data[i,:],train_data[j,:],kernel=self.kernel)       
        P = cvxopt.matrix(np.outer(train_label,train_label) * kernel)
        q = cvxopt.matrix(np.ones(train_num) * -1)
        A = cvxopt.matrix(train_label,(1,train_num),'d')
        b = cvxopt.matrix(0.0)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(train_num)*-1))
            h = cvxopt.matrix(np.zeros(train_num))
        else:
            M1 = np.diag(np.ones(train_num) * -1)
            M2 = np.identity(train_num)
            G = cvxopt.matrix(np.vstack([M1, M2]))
            M1 = np.zeros(train_num)
            M2 = np.ones(train_num) * self.C
            h = cvxopt.matrix(np.hstack([M1, M2]))

        solve = cvxopt.solvers.qp(P, q, G, h, A, b)
        a = np.ravel(solve['x'])
        sv = a > Epsilon     
        index = np.arange(len(a))[sv]  
        alpha = a[sv]
        SV = train_data[sv]  
        SV_label = train_label[sv]  
        omega = np.zeros(features)
        
        B = 0
        for n in range(len(alpha)):
            B = B + SV_label[n]
            B = B - np.sum(alpha*SV_label*kernel[index[n],sv])
        B = B / len(alpha)
            
        if self.kernel == 'Linear':
            for n in range(len(a)):
                omega += a[n] * train_label[n] * train_data[n]
            return (test_data@omega.T + B).reshape(-1,1)
        elif self.kernel == 'Gauss' or self.kernel == 'Poly':
            predict = np.zeros(len(test_data))
            for i in range(len(test_data)):
                y  = 0
                #for al,label,sv in zip(alpha,SV_label,SV):
                for al,label,sv in zip(a,train_label,train_data):
                    y += al * label * self.KERNEL(test_data[i],sv,kernel=self.kernel)
                predict[i] = y
            return predict.reshape(-1,1)
        else:
            print('No support for this kernel')
            return None

In [19]:
Train_data,Train_label,Test_data,Test_label=load_and_process_data()
Train_label=[label[0] for label in Train_label]
Test_label=[label[0] for label in Test_label]
train_data=np.array(Train_data)
test_data=np.array(Test_data)
test_label=np.array(Test_label).reshape(-1,1)
#类别个数
num_class=len(set(Train_label))

train_num: 3554
test_num: 983
train_feature's shape:(3554, 8)
test_feature's shape:(983, 8)


In [20]:
#kernel为核函数类型，可能的类型有'Linear'/'Poly'/'Gauss'
#C为软间隔参数；
#Epsilon为拉格朗日乘子阈值，低于此阈值时将该乘子设置为0
kernel='Linear' 
C = 1
Epsilon=10e-5
#生成SVM分类器
SVM=SupportVectorMachine(kernel,C,Epsilon)
predictions = []
#one-vs-all方法训练num_class个二分类器
for k in range(1,num_class+1):
    #将第k类样本label置为1，其余类别置为-1
    train_label=svm_label(Train_label,k)
    # 训练模型，并得到测试集上的预测结果
    prediction=SVM.fit(train_data,train_label,test_data)
    predictions.append(prediction)
predictions=np.array(predictions)
#one-vs-all, 最终分类结果选择最大score对应的类别
pred=np.argmax(predictions,axis=0)+1

# 计算准确率Acc及多分类的F1-score
print("Acc: "+str(get_acc(test_label,pred)))
print("macro-F1: "+str(get_macro_F1(test_label,pred)))
print("micro-F1: "+str(get_micro_F1(test_label,pred)))

     pcost       dcost       gap    pres   dres
 0: -1.4159e+03 -9.7614e+03  6e+04  3e+00  3e-13
 1: -9.4986e+02 -6.5633e+03  1e+04  4e-01  3e-13
 2: -9.0554e+02 -3.5160e+03  4e+03  1e-01  3e-13
 3: -9.5053e+02 -1.6024e+03  8e+02  3e-02  3e-13
 4: -1.0444e+03 -1.2923e+03  3e+02  8e-03  3e-13
 5: -1.0729e+03 -1.2298e+03  2e+02  4e-03  3e-13
 6: -1.0917e+03 -1.1902e+03  1e+02  2e-03  3e-13
 7: -1.1024e+03 -1.1692e+03  7e+01  1e-03  2e-13
 8: -1.1119e+03 -1.1517e+03  4e+01  7e-04  3e-13
 9: -1.1162e+03 -1.1438e+03  3e+01  4e-04  3e-13
10: -1.1203e+03 -1.1364e+03  2e+01  2e-04  3e-13
11: -1.1227e+03 -1.1328e+03  1e+01  1e-04  3e-13
12: -1.1246e+03 -1.1300e+03  6e+00  4e-05  3e-13
13: -1.1261e+03 -1.1280e+03  2e+00  6e-06  3e-13
14: -1.1266e+03 -1.1275e+03  9e-01  2e-06  3e-13
15: -1.1270e+03 -1.1270e+03  5e-02  6e-09  3e-13
16: -1.1270e+03 -1.1270e+03  2e-03  2e-10  3e-13
17: -1.1270e+03 -1.1270e+03  4e-05  3e-12  3e-13
Optimal solution found.
     pcost       dcost       gap    pres   dre

In [21]:
#kernel为核函数类型，可能的类型有'Linear'/'Poly'/'Gauss'
#C为软间隔参数；
#Epsilon为拉格朗日乘子阈值，低于此阈值时将该乘子设置为0
kernel='Gauss' 
C = 1
Epsilon=10e-5
#生成SVM分类器
SVM=SupportVectorMachine(kernel,C,Epsilon)
predictions = []
#one-vs-all方法训练num_class个二分类器
for k in range(1,num_class+1):
    #将第k类样本label置为1，其余类别置为-1
    train_label=svm_label(Train_label,k)
    # 训练模型，并得到测试集上的预测结果
    prediction=SVM.fit(train_data,train_label,test_data)
    predictions.append(prediction)
predictions=np.array(predictions)
#one-vs-all, 最终分类结果选择最大score对应的类别
pred=np.argmax(predictions,axis=0)+1

# 计算准确率Acc及多分类的F1-score
print("Acc: "+str(get_acc(test_label,pred)))
print("macro-F1: "+str(get_macro_F1(test_label,pred)))
print("micro-F1: "+str(get_micro_F1(test_label,pred)))

     pcost       dcost       gap    pres   dres
 0: -1.2803e+03 -8.7706e+03  5e+04  3e+00  3e-14
 1: -8.4676e+02 -5.4384e+03  7e+03  2e-01  3e-14
 2: -8.6401e+02 -1.5645e+03  8e+02  2e-02  2e-14
 3: -9.3835e+02 -1.3074e+03  4e+02  8e-03  2e-14
 4: -9.8443e+02 -1.1481e+03  2e+02  3e-03  2e-14
 5: -9.8781e+02 -1.1399e+03  2e+02  3e-03  2e-14
 6: -9.9845e+02 -1.1095e+03  1e+02  2e-03  2e-14
 7: -1.0076e+03 -1.0845e+03  8e+01  9e-04  2e-14
 8: -1.0143e+03 -1.0662e+03  5e+01  3e-04  2e-14
 9: -1.0183e+03 -1.0561e+03  4e+01  2e-04  2e-14
10: -1.0229e+03 -1.0436e+03  2e+01  1e-14  3e-14
11: -1.0260e+03 -1.0374e+03  1e+01  4e-15  2e-14
12: -1.0270e+03 -1.0353e+03  8e+00  2e-14  2e-14
13: -1.0281e+03 -1.0332e+03  5e+00  1e-14  2e-14
14: -1.0289e+03 -1.0318e+03  3e+00  4e-14  2e-14
15: -1.0295e+03 -1.0308e+03  1e+00  5e-14  3e-14
16: -1.0297e+03 -1.0306e+03  9e-01  3e-14  2e-14
17: -1.0297e+03 -1.0306e+03  9e-01  1e-13  2e-14
18: -1.0299e+03 -1.0304e+03  5e-01  3e-14  2e-14
19: -1.0299e+03 -1.03

In [22]:
#kernel为核函数类型，可能的类型有'Linear'/'Poly'/'Gauss'
#C为软间隔参数；
#Epsilon为拉格朗日乘子阈值，低于此阈值时将该乘子设置为0
kernel='Poly' 
C = 1
Epsilon=10e-5
#生成SVM分类器
SVM=SupportVectorMachine(kernel,C,Epsilon)
predictions = []
#one-vs-all方法训练num_class个二分类器
for k in range(1,num_class+1):
    #将第k类样本label置为1，其余类别置为-1
    train_label=svm_label(Train_label,k)
    # 训练模型，并得到测试集上的预测结果
    prediction=SVM.fit(train_data,train_label,test_data)
    predictions.append(prediction)
predictions=np.array(predictions)
#one-vs-all, 最终分类结果选择最大score对应的类别
pred=np.argmax(predictions,axis=0)+1

# 计算准确率Acc及多分类的F1-score
print("Acc: "+str(get_acc(test_label,pred)))
print("macro-F1: "+str(get_macro_F1(test_label,pred)))
print("micro-F1: "+str(get_micro_F1(test_label,pred)))

     pcost       dcost       gap    pres   dres
 0: -1.3453e+03 -1.0113e+04  6e+04  3e+00  2e-12
 1: -9.1009e+02 -6.9922e+03  1e+04  5e-01  2e-12
 2: -8.3485e+02 -3.6676e+03  4e+03  2e-01  2e-12
 3: -8.3788e+02 -2.2054e+03  2e+03  6e-02  2e-12
 4: -9.0835e+02 -1.3485e+03  5e+02  2e-02  2e-12
 5: -9.4480e+02 -1.2156e+03  3e+02  8e-03  2e-12
 6: -9.6805e+02 -1.1397e+03  2e+02  4e-03  2e-12
 7: -9.8196e+02 -1.0995e+03  1e+02  2e-03  2e-12
 8: -9.9427e+02 -1.0662e+03  8e+01  1e-03  2e-12
 9: -1.0006e+03 -1.0516e+03  5e+01  7e-04  2e-12
10: -1.0058e+03 -1.0395e+03  3e+01  4e-04  2e-12
11: -1.0091e+03 -1.0330e+03  2e+01  2e-04  2e-12
12: -1.0130e+03 -1.0253e+03  1e+01  8e-05  2e-12
13: -1.0145e+03 -1.0230e+03  9e+00  5e-05  2e-12
14: -1.0161e+03 -1.0203e+03  4e+00  1e-05  2e-12
15: -1.0170e+03 -1.0190e+03  2e+00  2e-06  2e-12
16: -1.0177e+03 -1.0183e+03  6e-01  3e-07  2e-12
17: -1.0179e+03 -1.0180e+03  8e-02  5e-08  2e-12
18: -1.0180e+03 -1.0180e+03  9e-03  4e-09  2e-12
19: -1.0180e+03 -1.01